In [ ]:
import sqlite3
import pandas as pd

In [ ]:
# Read the code below and write your observation in the next cell
conn = sqlite3.connect('subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master  WHERE type='table'")
print(cursor.fetchall())

In [ ]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

In [ ]:
df = pd.read_sql_query("""SELECT * FROM zipfiles limit 25000""", conn)
df.head()

In [ ]:
import zipfile
import io

# Assuming 'content' is the binary data from your database
binary_data = df.iloc[385, 2]

# Decompress the binary data using the zipfile module
with io.BytesIO(binary_data) as f:
    with zipfile.ZipFile(f, 'r') as zip_file:
        # Reading only one file in the ZIP archive
        subtitle_content = zip_file.read(zip_file.namelist()[0])

# Now 'subtitle_content' should contain the extracted subtitle content
print(subtitle_content.decode('latin-1'))  # Assuming the content is latin-1 encoded text


In [ ]:
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])

    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [ ]:
df['file_content'] = df['content'].apply(decode_method)

In [ ]:
df

remove timestamps

In [ ]:
import pandas as pd
df['file_content'] = df['file_content'].str.replace(r'\d+:\d+:\d+,\d+ --> \d+:\d+:\d+,\d+\r\n', '',regex=True)
df['file_content']

cleaning text

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords if not already downloaded
nltk.download('stopwords')

# Function to perform text cleaning
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Join tokens back into text
    cleaned_text = ' '.join(tokens)
    return cleaned_text

df['file_content'] = df['file_content'].apply(clean_text)
print(df)


# bert model

In [ ]:
df

In [ ]:
dff = df.head(100)
dff.to_csv('subtitle_data.csv')

In [ ]:
import pandas as pd
subtitle_content = pd.read_csv('subtitle_data.csv',index_col=0)
subtitle_content

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

# Load pre-trained BERT model for sentence embeddings
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Assuming 'file_content' is the column containing the text content of subtitle files
documents = subtitle_content['file_content'].tolist()

# Generate embeddings for the text content
document_embeddings = model.encode(documents)
print(document_embeddings)


In [ ]:
type(document_embeddings)

In [ ]:
subtitle_content

In [ ]:
subtitle_data = subtitle_content[['num', 'name', 'file_content']]
subtitle_data

In [ ]:
type(subtitle_data)

# chromadb

In [ ]:
import sqlite3
import json

# Connect to SQLite database
conn = sqlite3.connect('chroma_embeddings.db')
cursor = conn.cursor()

# Create subtitles table if not exists
cursor.execute('''CREATE TABLE IF NOT EXISTS subtitles (
                    num INTEGER PRIMARY KEY,
                    name TEXT NOT NULL,
                    file_content TEXT NOT NULL,
                    embedding TEXT
                )''')

# Commit changes
conn.commit()

# Insert subtitle data and embeddings into the subtitles table
for idx, row in subtitle_data.iterrows():
    num = row['num']
    name = row['name']
    file_content = row['file_content']
    embedding = json.dumps(document_embeddings[idx].tolist())  # Retrieve embedding using index
    
    try:
        cursor.execute('''INSERT INTO subtitles (num, name, file_content, embedding)
                          VALUES (?, ?, ?, ?)''', (num, name, file_content, embedding))
    except sqlite3.IntegrityError:
        # Handle duplicate num values here, such as skipping the insertion or updating existing rows
        pass

# Commit changes and close connection
conn.commit()
conn.close()


In [ ]:
import sqlite3
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json

def retrieve_documents(query, database_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Fetch names and embeddings from the database
    cursor.execute('''SELECT name, embedding FROM subtitles''')
    rows = cursor.fetchall()

    # Extract names and embeddings from rows
    document_names = [row[0] for row in rows]
    embeddings = [np.array(json.loads(row[1])) for row in rows]

    # Close the database connection
    conn.close()

    # Load the BERT model
    model = SentenceTransformer('bert-base-nli-mean-tokens')

    # Encode the query using the BERT model
    query_embedding = model.encode([query])[0]

    # Calculate cosine similarity between query embedding and document embeddings
    similarities = cosine_similarity([query_embedding], embeddings)

    # Rank documents based on similarity scores
    ranked_indices = np.argsort(similarities[0])[::-1]  # Descending order

    # Return ranked documents
    ranked_documents = [(document_names[idx], similarities[0][idx]) for idx in ranked_indices]
    return ranked_documents

# Example usage
query = "search query"
database_path = 'chroma_embeddings.db'
ranked_documents = retrieve_documents(query, database_path)